In [ ]:
#Apply model on images and store it with 4 classes(Vest, NOVest, Helmet, NOHelmet)

from ultralytics import YOLO

import os
from PIL import Image, ImageDraw, ImageFont

def draw_text_and_box_on_image(image, text, position, box_coordinates, font, color):
    draw = ImageDraw.Draw(image)
    
    # Draw bounding box
    draw.rectangle(box_coordinates, outline=color, width=2)
    
    # Draw text
    draw.text(position, text, fill=color, font=font)
    
    return image

def process_images(folder_path, model, output_folder):
    # List all files in the specified folder
    files = os.listdir(folder_path)

    # Filter only image files (you can customize this based on your image file extensions)
    image_files = [f for f in files if f.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp'))]
    
    for image_file in image_files:
        # Construct the full path to the image
        image_path = os.path.join(folder_path, image_file)
        # Open the image using Pillow
        try:
            results = model.predict(image_path)
            result = results[0]
            # Create a font with the desired size
            font_size = 10  # Adjust the font size as needed
            font = ImageFont.truetype("arial.ttf", font_size)  # Use arial.ttf or another font file with the desired size
            original_image = Image.open(image_path)
            
            for idx, box in enumerate(result.boxes):
                class_id = result.names[box.cls[0].item()]
                cords = box.xyxy[0].tolist()
                cords = [round(x) for x in cords]
                conf = round(box.conf[0].item(), 1)
                
                # Determine color and text based on class_id
                if class_id == "Helmet":
                    bounding_box_color = (0, 255, 0)  # Green
                    text_color = (0, 255, 0)  # Green
                elif class_id == "NOHelmet":
                    bounding_box_color = (0, 0, 255)  # Blue
                    text_color = (0, 0, 255)  # Blue
                elif class_id == "NOVest":
                    bounding_box_color = (255, 0, 0)  # Red
                    text_color = (255, 0, 0)  # Red
                elif class_id == "Vest":
                    bounding_box_color = (255, 255, 0)  # Yellow
                    text_color = (255, 255, 0)  # Yellow
                else:
                    # Default color for unknown classes
                    bounding_box_color = (128, 128, 128)  # Gray
                    text_color = (128, 128, 128)  # Gray

                # Draw text and bounding box on the image
                text = f"{class_id}({conf})"
                position = (cords[0], cords[1] - 12)  # Adjust the position based on your preference
                image_with_text_and_box = draw_text_and_box_on_image(original_image, text, position, cords, font, bounding_box_color)
                
            # Save the modified image to the output folder outside the loop
            output_path = os.path.join(output_folder, f'modified_image_{image_file.replace(".", "_")}.png')  # Use a unique identifier for each image
            image_with_text_and_box.save(output_path, format='PNG')

        except Exception as e:
            print(f"Error processing image {image_path}: {e}")

# Example usage
output_folder = "Your output folder path"
model = YOLO("Your Model path")
folder_path = "Your Input folder path"
process_images(folder_path, model, output_folder)

In [ ]:
#Code to apply model on video with 4 classes(Vest, NOVest, Helmet, NOHelmet)

from ultralytics import YOLO
import os
from PIL import Image, ImageDraw, ImageFont
import cv2
import numpy as np

def draw_text_and_box_on_image(image, text, position, box_coordinates, font, color):
    draw = ImageDraw.Draw(image)
    
    # Draw bounding box
    draw.rectangle(box_coordinates, outline=color, width=2)
    
    # Draw text with specified color
    draw.text(position, text, fill=color, font=font)
    
    return image

def process_video(video_path, model, output_folder, output_video_path):
    # Open the video capture
    cap = cv2.VideoCapture(video_path)
    
    # Get the frames per second (fps) of the input video
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Create output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Create VideoWriter object with the same fps as the input video
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (int(cap.get(3)), int(cap.get(4))))

    frame_number = 0

    while True:
        # Read a frame from the video
        ret, frame = cap.read()

        # Break the loop if the video has ended
        if not ret:
            break
        
        # Convert the OpenCV BGR image to RGB (PIL format)
        image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

        try:
            results = model.predict(image)
            result = results[0]

            # Create a font with the desired size
            font_size = 20  # Adjust the font size as needed
            font = ImageFont.truetype("arial.ttf", font_size)  # Use arial.ttf or another font file with the desired size

            # Keep track of whether any detection occurred in this frame
            detection_occurred = False

            for idx, box in enumerate(result.boxes):
                class_id = result.names[box.cls[0].item()]
                cords = box.xyxy[0].tolist()
                cords = [round(x) for x in cords]
                conf = round(box.conf[0].item(), 2)

                # Determine color based on class_id
                if class_id == "Helmet":
                    bounding_box_color = (0, 255, 0)  # Green
                    text_color = (0, 255, 0)  # Green
                elif class_id == "NOHelmet":
                    bounding_box_color = (0, 0, 255)  # Blue
                    text_color = (0, 0, 255)  # Blue
                elif class_id == "NOVest":
                    bounding_box_color = (255, 0, 0)  # Red
                    text_color = (255, 0, 0)  # Red
                elif class_id == "Vest":
                    bounding_box_color = (255, 255, 0)  # Yellow
                    text_color = (255, 255, 0)  # Yellow
                else:
                    bounding_box_color = (128, 128, 128)  # Default to gray
                    text_color = (128, 128, 128)  # Default to gray

                # Draw text and bounding box on the image
                text = f"{class_id}({conf})"
                position = (cords[0], cords[1] - 22)  # Adjust the position based on your preference
                image_with_text_and_box = draw_text_and_box_on_image(image, text, position, cords, font, text_color)

                # Convert the modified image back to BGR (OpenCV format)
                modified_frame = cv2.cvtColor(np.array(image_with_text_and_box), cv2.COLOR_RGB2BGR)

                # If at least one detection occurred, set detection_occurred to True
                if not detection_occurred:
                    detection_occurred = True

            # Write the frame to the output video (outside the detection loop)
            out.write(modified_frame) if detection_occurred else out.write(frame)

        except Exception as e:
            print(f"Error processing frame {frame_number}: {e}")

        frame_number += 1

    # Release the video capture and writer
    cap.release()
    out.release()

# Example usage
output_folder = "Your output folder path"
output_video_path = "Your output video path"
model = YOLO("Your Model path")
video_path = "Your Input video path"
process_video(video_path, model, output_folder, output_video_path)


In [ ]:
# Model to apply on webcam and show output for 4 classes(Vest, NOVest, Helmet, NOHelmet)

from ultralytics import YOLO
import cv2
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import time

def draw_text_and_box_on_image(image, text, position, box_coordinates, font, color):
    draw = ImageDraw.Draw(image)
    
    # Draw bounding box
    draw.rectangle(box_coordinates, outline=color, width=2)
    
    # Draw text with specified color
    draw.text(position, text, fill=color, font=font)
    
    return image

def process_webcam(model, timeout=5):
    # Open the webcam
    cap = cv2.VideoCapture(0)  # 0 corresponds to the default webcam

    # Create a font with the desired size
    font_size = 20
    font = ImageFont.truetype("arial.ttf", font_size)

    # Initialize timeout variables
    last_detection_time = time.time()

    while True:
        # Read a frame from the webcam
        ret, frame = cap.read()

        # Convert the OpenCV BGR image to RGB (PIL format)
        image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

        try:
            results = model.predict(image)
            result = results[0]

            # Create a flag to track if detection occurred in the current frame
            detection_occurred = False

            for idx, box in enumerate(result.boxes):
                class_id = result.names[box.cls[0].item()]
                cords = box.xyxy[0].tolist()
                cords = [round(x) for x in cords]
                conf = round(box.conf[0].item(), 2)

                # Determine color based on class_id
                if class_id == "Helmet":
                    bounding_box_color = (0, 255, 0)  # Green
                    text_color = (0, 255, 0)  # Green
                elif class_id == "NOHelmet":
                    bounding_box_color = (0, 0, 255)  # Blue
                    text_color = (0, 0, 255)  # Blue
                elif class_id == "NOVest":
                    bounding_box_color = (255, 0, 0)  # Red
                    text_color = (255, 0, 0)  # Red
                elif class_id == "Vest":
                    bounding_box_color = (255, 255, 0)  # Yellow
                    text_color = (255, 255, 0)  # Yellow
                else:
                    bounding_box_color = (128, 128, 128)  # Default to gray
                    text_color = (128, 128, 128)  # Default to gray

                # Draw text and bounding box on the image
                text = f"{class_id}({conf})"
                position = (cords[0], cords[1] - 22)  # Adjust the position based on your preference
                image_with_text_and_box = draw_text_and_box_on_image(image, text, position, cords, font, text_color)

                # Convert the modified image back to BGR (OpenCV format)
                modified_frame = cv2.cvtColor(np.array(image_with_text_and_box), cv2.COLOR_RGB2BGR)

                # Set the flag to True if at least one detection occurs
                detection_occurred = True

                # Update the last detection time
                last_detection_time = time.time()

            # Show the modified frame on the screen only if detection occurred
            if detection_occurred:
                cv2.imshow('Webcam Detection', modified_frame)
            else:
                # Show the original frame when no detection occurs
                cv2.imshow('Webcam Detection', frame)

        except Exception as e:
            print(f"Error processing frame: {e}")

        # Break the loop when 'q' key is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the webcam
    cap.release()
    cv2.destroyAllWindows()

model = YOLO("Your Model path")
process_webcam(model)


In [ ]:
# Model to apply on video and get output of safety/unsafety percentages for Helmet and Vest detection.

import os
from ultralytics import YOLO
import cv2

def apply_safety_detection(video):
    # Code to apply safety detection model on the video
    model = YOLO("Your Model path")

    # Open the video capture
    cap = cv2.VideoCapture(str(video))
    
    # Get the frames per second (fps) of the input video
    fps = cap.get(cv2.CAP_PROP_FPS)

    frame_number = 0

    # Initialize counts for safety and unsafety
    safety_count = 0
    unsafety_count = 0

    while True:
        # Read a frame from the video
        ret, frame = cap.read()

        # Break the loop if the video has ended
        if not ret:
            break

        try:
            results = model.predict(frame)  # Assuming the model can directly predict on OpenCV frame
            result = results[0]

            # Update counts for safety and unsafety based on detections
            safety_classes = ["Vest", "Helmet"]  # Adjust this list based on your model's class names
            for idx, box in enumerate(result.boxes):
                class_id = result.names[box.cls[0].item()]
                conf = box.conf[0].item()

                if class_id in safety_classes:  # and conf >= 0.5
                    safety_count += 1
                else:
                    unsafety_count += 1

        except Exception as e:
            print(f"Error processing frame {frame_number}: {e}")

        frame_number += 1

    # Release the video capture
    cap.release()
    
    # Calculate percentages based on the total number of detections
    total_detections = safety_count + unsafety_count

    # Avoid division by zero
    safety_percentage = (safety_count / total_detections) * 100 if total_detections > 0 else 0
    unsafety_percentage = (unsafety_count / total_detections) * 100 if total_detections > 0 else 0

    return str(video), safety_percentage, unsafety_percentage

apply_safety_detection("Your Input video path")


In [ ]:
# Person detection in Restricted area

import cv2
from ultralytics import YOLO
import numpy as np

model = YOLO("Yout Model path")
cap = cv2.VideoCapture("Your Input video path")

exit_delay_counter = 0
person_inside_polygon = False
processed_persons = []

fps = cap.get(cv2.CAP_PROP_FPS)
desired_delay_seconds = 0.3
delay_frames = int(fps * desired_delay_seconds)

def is_point_inside_polygon(point, polygon):
    if len(polygon) >= 2:
        result = cv2.pointPolygonTest(polygon, point, False)
        return result > 0
    return False

drawing = False
polygon_points = []

def draw_polygon(event, x, y, flags, param):
    global drawing, polygon_points

    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        polygon_points.append((x, y))
    elif event == cv2.EVENT_LBUTTONUP and drawing:
        polygon_points.append((x, y))
        poly = cv2.polylines(frame, [np.array(polygon_points)], isClosed=True, color=(0, 0, 255), thickness=2)

    elif event == cv2.EVENT_RBUTTONDOWN and drawing:
        drawing = False
        cv2.polylines(frame, [np.array(polygon_points)], isClosed=True, color=(0, 0, 255), thickness=2)
        polygon_points = []

cv2.namedWindow('video')
cv2.setMouseCallback('video', draw_polygon)

while True:
    ret, frame = cap.read()
    frame = cv2.resize(frame, (640, 480))

    result = model.predict(frame, classes=[0])
    frame = result[-1].plot(conf=False)

    new_person_inside_polygon = False

    for box in result[-1].boxes.data:
        class_id = int(box[5])
        if class_id == 0:
            person_bbox = box[:4].cpu().numpy()

            x, y, w, h = person_bbox
            cv2.rectangle(frame, tuple(map(int, person_bbox[:2])), tuple(map(int, person_bbox[2:])), (0, 255, 0), 2)

            person_center_x = (x + w) / 2
            person_center_y = (y + h) / 2

            center = (int(person_center_x), int(person_center_y))
            cv2.circle(frame, center, 5, (0, 255, 255), -1)

            top_right_corner = (person_bbox[2], person_bbox[1])
            top_left_corner = (person_bbox[0], person_bbox[1])
            bottom_left_corner = (person_bbox[0], person_bbox[3])
            bottom_right_corner = (person_bbox[2], person_bbox[3])

            intersection_area = is_point_inside_polygon(bottom_left_corner, np.array(polygon_points)) or is_point_inside_polygon(bottom_right_corner, np.array(polygon_points)) or is_point_inside_polygon(top_left_corner, np.array(polygon_points)) or is_point_inside_polygon(top_right_corner, np.array(polygon_points)) or is_point_inside_polygon(center, np.array(polygon_points))

            if intersection_area and person_bbox.tolist() not in processed_persons:
                cv2.rectangle(frame, tuple(map(int, person_bbox[:2])), tuple(map(int, person_bbox[2:])), (255, 0, 0), 2)
                new_person_inside_polygon = True
                processed_persons.append(person_bbox.tolist())

    if not person_inside_polygon and new_person_inside_polygon:
        person_inside_polygon = True
        new_person_inside_polygon = False

    elif person_inside_polygon and not new_person_inside_polygon:
        exit_delay_counter += 1
        if exit_delay_counter >= delay_frames:
            person_inside_polygon = False
            exit_delay_counter = 0

    if person_inside_polygon:
        cv2.polylines(frame, [np.array(polygon_points)], isClosed=True, color=(0, 0, 255), thickness=2)
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(frame, 'person entered restricted area', (10, 30), font, 0.8, (255, 255, 255), 2)
    else:
        cv2.polylines(frame, [np.array(polygon_points)], isClosed=True, color=(255, 255, 255), thickness=2)

    cv2.imshow("video", frame)
    if cv2.waitKey(1) == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()
